# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 6276, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 6276 (delta 65), reused 25 (delta 10), pack-reused 6089 (from 4)
Receiving objects: 100% (6276/6276), 401.57 MiB | 37.20 MiB/s, done.
Resolving deltas: 100% (3445/3445), done.
Updating files: 100% (490/490), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.4 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprecated NumPy API, disabl

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithKFCV/GraphBased'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'RP3beta',
    'metric': 'MAP',
    'n_folds': 5,
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_RP3beta_MAP.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_k_folds import split_train_k_folds

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

folds = split_train_k_folds(URM_all, k=config['n_folds'])

In [12]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

def objective_function_RP3beta(optuna_trial):
    
    recommender_instance = RP3betaRecommender(URM_train)
    
    full_hyperp = {
                    "topK": optuna_trial.suggest_int("topK", 0, 1500),
                   "alpha": optuna_trial.suggest_float("alpha", 0, 3, log=False),
                   "beta": optuna_trial.suggest_float("beta", 0, 3, log=False),
                   "normalize_similarity": optuna_trial.suggest_categorical("normalize_similarity", [True, False]),
                   "implicit": optuna_trial.suggest_categorical("implicit", [True, False]),
                  }            

    validation_results = []
    
    for fold_idx, (URM_train_fold, URM_validation_fold) in enumerate(folds):
    
        recommender_instance = RP3betaRecommender(URM_train_fold)
        recommender_instance.fit(**full_hyperp)
        
        evaluator = EvaluatorHoldout(URM_validation_fold, cutoff_list=[10])
        result_df, _ = evaluator.evaluateRecommender(recommender_instance)
        
        validation_results.append(result_df.loc[10]["MAP"])
    
    return np.mean(validation_results)

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_RP3beta, n_trials=50)

[I 2025-01-05 14:59:29,889] Using an existing study with name 'hyperparameters_tuning_RP3beta_MAP' instead of creating a new one.


RP3betaRecommender: Similarity column 38121 (100.0%), 1874.64 column/sec. Elapsed time 20.34 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 35.04 sec. Users per second: 1015
RP3betaRecommender: Similarity column 38121 (100.0%), 1876.70 column/sec. Elapsed time 20.31 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 34.84 sec. Users per second: 1021
RP3betaRecommender: Similarity column 38121 (100.0%), 1883.14 column/sec. Elapsed time 20.24 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 34.71 sec. Users per second: 1025
RP3betaRecommender: Similarity column 38121 (100.0%), 1870.24 column/sec. Elapsed time 20.38 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 3556

[I 2025-01-05 15:04:23,482] Trial 200 finished with value: 0.051329600467651816 and parameters: {'topK': 70, 'alpha': 0.3659450195566235, 'beta': 0.387955097433834, 'normalize_similarity': True, 'implicit': False}. Best is trial 154 with value: 0.05406959471293592.


RP3betaRecommender: Similarity column 38121 (100.0%), 2129.25 column/sec. Elapsed time 17.90 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 32.11 sec. Users per second: 1108
RP3betaRecommender: Similarity column 38121 (100.0%), 2082.09 column/sec. Elapsed time 18.31 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 32.08 sec. Users per second: 1109
RP3betaRecommender: Similarity column 38121 (100.0%), 2119.12 column/sec. Elapsed time 17.99 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 32.03 sec. Users per second: 1111
RP3betaRecommender: Similarity column 38121 (100.0%), 2138.74 column/sec. Elapsed time 17.82 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 3556

[I 2025-01-05 15:08:42,743] Trial 201 finished with value: 0.05428822463060982 and parameters: {'topK': 27, 'alpha': 0.42917739971377455, 'beta': 0.28803459854596214, 'normalize_similarity': True, 'implicit': False}. Best is trial 201 with value: 0.05428822463060982.


...

[I 2025-01-05 18:45:07,983] Trial 247 finished with value: 0.036904562559433474 and parameters: {'topK': 2, 'alpha': 0.5724097023743501, 'beta': 0.14515494524773498, 'normalize_similarity': True, 'implicit': True}. Best is trial 229 with value: 0.0549701405549974.


RP3betaRecommender: Similarity column 38121 (100.0%), 2159.42 column/sec. Elapsed time 17.65 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 31.55 sec. Users per second: 1127
RP3betaRecommender: Similarity column 38121 (100.0%), 2141.97 column/sec. Elapsed time 17.80 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 31.49 sec. Users per second: 1130
RP3betaRecommender: Similarity column 38121 (100.0%), 2128.42 column/sec. Elapsed time 17.91 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 31.48 sec. Users per second: 1130
RP3betaRecommender: Similarity column 38121 (100.0%), 2134.84 column/sec. Elapsed time 17.86 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 3556

[I 2025-01-05 18:49:22,273] Trial 248 finished with value: 0.05395554901516657 and parameters: {'topK': 25, 'alpha': 0.598427396397135, 'beta': 0.1179279564123352, 'normalize_similarity': True, 'implicit': True}. Best is trial 229 with value: 0.0549701405549974.


RP3betaRecommender: Similarity column 38121 (100.0%), 2917.07 column/sec. Elapsed time 13.07 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 24.51 sec. Users per second: 1451
RP3betaRecommender: Similarity column 38121 (100.0%), 2911.24 column/sec. Elapsed time 13.09 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 24.37 sec. Users per second: 1460
RP3betaRecommender: Similarity column 38121 (100.0%), 2913.46 column/sec. Elapsed time 13.08 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 24.31 sec. Users per second: 1464
RP3betaRecommender: Similarity column 38121 (100.0%), 2920.00 column/sec. Elapsed time 13.06 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 3556

[I 2025-01-05 18:52:30,773] Trial 249 finished with value: 0.0002664826418719655 and parameters: {'topK': 0, 'alpha': 0.5841476261828616, 'beta': 0.09215455324782836, 'normalize_similarity': True, 'implicit': True}. Best is trial 229 with value: 0.0549701405549974.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = RP3betaRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

RP3betaRecommender: Similarity column 38121 (100.0%), 1959.68 column/sec. Elapsed time 19.45 sec


# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_RP3beta_MAP.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithKFCV/GraphBased/RP3betaRecommender/OptimizingMAP/best_params_RP3beta_MAP.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithKFCV/GraphBased/RP3betaRecommender/OptimizingMAP/history_RP3beta_MAP.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithKFCV/GraphBased/RP3betaRecommender/OptimizingMAP/Submission/submission_RP3beta_MAP.csv' updated successfully.
